In [1]:
import numpy as np
import pybullet as p
import pybullet_data
import time

In [2]:
p.connect(p.GUI)
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(0)

In [16]:
p.loadURDF("plane.urdf",[0,0,0],[0,0,0,1], useFixedBase = True)
targid = p.loadURDF("franka_panda/panda.urdf",[0,0,0],[0,0,0,1], useFixedBase = True)
obj_of_focus = targid

In [7]:
for i in range(p.getNumJoints(targid)):
    print(p.getJointInfo(targid,i))

(0, b'panda_joint1', 0, 7, 6, 1, 0.0, 0.0, -2.9671, 2.9671, 87.0, 2.175, b'panda_link1', (0.0, 0.0, 1.0), (0.0, 0.0, 0.28300000000000003), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'panda_joint2', 0, 8, 7, 1, 0.0, 0.0, -1.8326, 1.8326, 87.0, 2.175, b'panda_link2', (0.0, 0.0, 1.0), (0.0, 0.04, 0.05), (0.7071067811848163, 0.0, 0.0, 0.7071067811882787), 0)
(2, b'panda_joint3', 0, 9, 8, 1, 0.0, 0.0, -2.9671, 2.9671, 87.0, 2.175, b'panda_link3', (0.0, 0.0, 1.0), (0.0, -0.276, -0.06), (-0.7071067811848163, 0.0, 0.0, 0.7071067811882787), 1)
(3, b'panda_joint4', 0, 10, 9, 1, 0.0, 0.0, -3.1416, 0.0, 87.0, 2.175, b'panda_link4', (0.0, 0.0, 1.0), (0.07250000000000001, -0.01, 0.05), (-0.7071067811848163, 0.0, 0.0, 0.7071067811882787), 2)
(4, b'panda_joint5', 0, 11, 10, 1, 0.0, 0.0, -2.9671, 2.9671, 12.0, 2.61, b'panda_link5', (0.0, 0.0, 1.0), (-0.052500000000000005, 0.354, -0.02), (0.7071067811848163, 0.0, 0.0, 0.7071067811882787), 3)
(5, b'panda_joint6', 0, 12, 11, 1, 0.0, 0.0, -0.0873, 3.8223, 12.0, 2.61, 

In [4]:
jointId = 4
jtype = p.getJointInfo(targid, jointId)[2] #type of joint
jlower = p.getJointInfo(targid, jointId)[8] #minimum angle of the joint
jupper = p.getJointInfo(targid, jointId)[9] #maximum angle of the joint


In [5]:
for step in range(300):
    joint_two_targ = np.random.uniform(jlower,jupper)
    joint_four_targ = np.random.uniform(jlower,jupper)
    p.setJointMotorControlArray(targid, [2,4], p.POSITION_CONTROL, targetPositions = [joint_two_targ, joint_four_targ])
    focus_position,_ = p.getBasePositionAndOrientation(targid)
    p.resetDebugVisualizerCamera(cameraDistance=3, cameraYaw=0, cameraPitch=-40, cameraTargetPosition = focus_position)
    p.stepSimulation()
    time.sleep(0.1)

In [17]:
jointId = 4

jlower = p.getJointInfo(targid, jointId)[8]
jupper = p.getJointInfo(targid, jointId)[9]

In [22]:
class ArmEnv():
    def __init__(self):
        self.state = self.init_state()
        self.step_count = 0

    def init_state(self):
        p.connect(p.DIRECT)
        p.resetSimulation()
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0,0,-9.87)
        self.pandaUid = p.loadURDF("Franka_panda/panda.urdf", [0,0,0],[0,0,0,1],useFixedBase=True)
        p.loadURDF("plane.urdf",[0,0,0],[0,0,0,1])
        finger_pos = p.getLinkState(self.pandaUid, 9)[0]
        obs = np.array([finger_pos]).flatten()

    def reset(self):
        p.disconnect()
        self.state = self.init_state()
        self.step_count = 0

    def step(self, action):
        self.step_count += 1
        p.setJointMotorControlArray(self.pandaUid, [4], p.POSITION_CONTROL, [action])
        p.stepSimulation()
        finger_pos = p.getLinkState(self.pandaUid, 9)[0]
        
        if(self.step_count >=50):
            self.reset()
            finger_pos = p.getLinkState(self.pandaUid, 9)[0]
            
            obs = np.array([finger_pos]).flatten()
            self.state = obs
            reward = -1 #arbitrary reward
            done = True
            return reward, done
   
        obs = np.array([finger_pos]).flatten()
        self.state = obs
        reward = -1 #arbitrary reward
        done = True
        return reward, done




In [23]:
env =   ArmEnv()
for step in range(500):
    action = np.random.uniform(jlower, jupper) #new position
    a,b = env.step(action)
    print(env.state)
    p.stepSimulation()


[ 0.09507107 -0.00707107  0.8476    ]
[ 0.09519338 -0.00517737  0.84759997]
[ 0.09520324 -0.00498175  0.84760001]
[0.09487866 0.0187963  0.84734072]
[0.09382923 0.01407645 0.84768397]
[ 0.09214986 -0.01161842  0.84804864]
[ 0.08607782 -0.0353199   0.84801942]
[ 0.06060585 -0.07203899  0.84781949]
[ 0.06360907 -0.06977788  0.84776846]
[ 0.04870109 -0.08114168  0.84772961]
[ 0.06743963 -0.06595892  0.84778479]
[ 0.07003811 -0.06313772  0.84779179]
[ 0.05985755 -0.07305717  0.84776426]
[ 0.08035746 -0.04904878  0.84781962]
[ 0.07799768 -0.05236152  0.84782732]
[ 0.05042997 -0.08009299  0.84769389]
[ 0.04066243 -0.08562629  0.84767192]
[ 0.06463037 -0.06861157  0.84777352]
[ 0.05003585 -0.0802182   0.84771915]
[ 0.06109218 -0.07194227  0.8477511 ]
[ 0.06993922 -0.06318192  0.84777606]
[ 0.06881542 -0.06443233  0.84777306]
[ 0.06604684 -0.06732746  0.8477655 ]
[ 0.06065934 -0.07232585  0.8477507 ]
[ 0.05116056 -0.07949988  0.8477246 ]
[ 0.07331608 -0.05915874  0.84778564]
[ 0.05774802 -0.07